<a href="https://colab.research.google.com/github/bloodclaw2000/Dementia_OASIS_Saturdays/blob/main/dementia%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
sys.path.insert(0, '/usr/local/lib/python3.11/dist-packages/') #We need to upgrade Python Client for some reason COLAB is py 3.6

#!python -m pip install --upgrade pip
!git clone https://github.com/bloodclaw2000/Dementia_OASIS_Saturdays.git
%cd Dementia_OASIS_Saturdays

Cloning into 'Dementia_OASIS_Saturdays'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 66 (delta 11), reused 47 (delta 9), pack-reused 10
Receiving objects: 100% (66/66), 135.34 MiB | 25.07 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Dementia_OASIS_Saturdays


In [2]:
%mkdir nn
%mkdir dataset


In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import sys
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import pickle
import _pickle as cPickle
import bz2
import csv
import torch
import torchvision
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
import random
from torchvision import transforms
from tabulate import tabulate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from IPython.display import Image
from sklearn import tree

In [5]:
from pickle_aux import pet_load, decompress_pickle, pet_save
from dementia_network_class import Dementia, train_nn, getOutput
from dementia_tree_class import train_tree,  Customtree



In [6]:
torch.multiprocessing.set_start_method('spawn', force=True) #kinda important mostly for CPU

device = torch.device(
                                        f'cuda:{torch.cuda.current_device()}'
                                        if torch.cuda.is_available()
                                        else 'cpu')

In [7]:
compressed_pickle_directory = "save_dict3"
if not os.path.exists('{0}_decompressed.p'.format(compressed_pickle_directory)):
    def force_dementia(dictionary):
              for key in dictionary:
                        for key2 in dictionary[key]:
                            if dictionary[key][key2]['CDR'] == '':
                                      dictionary[key][key2]['Dementia'] = 0
                            elif float(dictionary[key][key2]['CDR']) > 0:
                                      dictionary[key][key2]['Dementia'] = 1
                            else:
                                      dictionary[key][key2]['Dementia'] = 0
              return dictionary

    def removeyoung(dictionary, age):
              dic_pacientes_viejos = {}
              for key in dictionary:
                        for key2 in dictionary[key]:
                            if int(dictionary[key][key2]['Age']) >= age:
                                      dic_pacientes_viejos[key] = dictionary[key]
              return dic_pacientes_viejos

    tmp_dict = decompress_pickle('{0}.pbz2'.format(compressed_pickle_directory))
    tmp_dict = force_dementia(tmp_dict) #esto es la funcion del init
    tmp_dict = removeyoung(tmp_dict, 59)
    tmp_dict = {int(key): value for key, value in tmp_dict.items()}
    new_dict = {}
    current_index = 0
    for key in sorted(tmp_dict.keys()):
                        new_dict[current_index] = tmp_dict[key]
                        current_index += 1
    tmp_dict = new_dict

    pet_save(tmp_dict,'{0}_decompressed.p'.format(compressed_pickle_directory))
else:
     tmp_dict=pet_load('{0}_decompressed.p'.format(compressed_pickle_directory))


In [8]:
torch.set_default_device(device)

print(device)

print(f" Using {device} in this run")

obj = Dementia(dictionary=tmp_dict,device=device)


cpu
 Using cpu in this run


In [9]:
#Podemos setear los parámetros para el entrenamiento aquí:
obj.setParam('image_type','T88_111')
obj.setParam('image_number',1)


In [ ]:
# train(obj,'params/',['T88'])
# train(obj,'params/',['T88','FSL'])
train_nn(obj, 'params_nn/', ['T88', 'FSL', 'RAW_1', 'RAW_2', 'RAW_3'])

# getOutput('nn/',['T88'])
# getOutput('nn/',['T88','FSL'])
getOutput('nn/', ['T88', 'FSL', 'RAW_1', 'RAW_2', 'RAW_3'])


Read parameters dictionary at relative location params_nn/RAW_1.txt and setting them 
Param invalid_key does not exist
Param does not exist
Resetting to default value
Now calculating predictions for file params_nn/RAW_1.txt
{'plot': 'True', 'image_type': 'RAW', 'image_number': 1, 'patience_validation': 3, 'patience_plateau': 3, 'validation_patience': 3, 'delta_min': 0, 'batch_size': 10, 'split_size': 0.8, 'max_loss_reset': 5, 'learning_rate': 0.0001, 'weight_decay': 0.1, 'nepochs': 1000, 'first_conv_outchann': 6, 'second_conv_outchann': 16, 'fclayer1': 120, 'fclayer2': 'None', 'criterion_type': 'BCElogitsloss', 'optimizer': 'Adam'}


In [ ]:

source_df = pd.read_csv('results.csv')
treeclass = Customtree(source_df)
#treeclass.write_dict_to_file('params_tree/treeparam.txt')
train_tree(treeclass,'params_tree/treeparam.txt')
